Importing Libraries

In [238]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch, pandas as pd
import subprocess
import os


In [239]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

Loading datasets

In [263]:
class LlmBaseline:
    def __init__(self, models=None, df=None):
        self.models = models if models else []
        self.dataframes = df if df else []
        self.summaries = []

    def add_df(self, df):
        self.dataframes.append(df)

    def read_dataset_path(self, path):
        self.dataframes.append(pd.read_csv(path))

    def get_all_df(self):
        return self.dataframes

    def generate_summary(self, model_name, prompt):
        # Send the prompt to model via Ollama (no model loaded in memory)
        result = subprocess.run(
            ["ollama", "run", model_name],
            input=prompt,
            text=True,
            capture_output=True
        )
        
        return result.stdout.strip()

    def generate_promt(self, data):
        prompt = f"""
            Based on the given data:
            {data}
            Your task is to write a brief, fluent,
            and coherent single-paragraph summary
            in natural language. The text should be
            balanced and neutral. Make sure that all the
            facts mentioned in the text can be derived
            from the input data, do *not* add any extra
            information.
        """
    
        return prompt

    # Convert the dataframe to readable formats i.e. csv and json
    def data_converter_csv(self, df):
        return df.to_csv(index=False)
    
    def data_converter_json(self, df):
        return df.reset_index().to_json(orient='records')

    def generator(self):
        if self.models == []:
            print("No models selected")
            return
        if self.dataframes == []:
            print("No dataframes added")
            return

        for df in self.dataframes:
            data = self.data_converter_json(df)
            prompt = self.generate_promt(data)
            summary = []
            
            for model in self.models:
                summary.append({"model": model, "summary": self.generate_summary(model, prompt)})

            self.summaries.append({"data": data, "summaries": summary})

        return self.summaries

    def save_summaries(self):
        if self.summaries == []:
            print("No summaries generated")
            return

        for i, summary in enumerate(summaries):
            for item in summary['summaries']:
                filename = f"summary_{i+1}_{item['model']}.txt"   # Creates file_1.txt, file_2.txt, etc.
                with open(filename, "w") as f:
                    f.write(item['summary'])
                print(f"Saved {filename}")

In [241]:
current_directory = os.getcwd()
print(f"Current working directory: {current_directory}")

Current working directory: /Users/muhammadasad/Desktop/Data Mining/Project


In [242]:
# Load your dataset
df2010_2014 = pd.read_csv(current_directory + "/Datasets/2010_2014.csv")

df2015_2019 = pd.read_csv(current_directory + "/Datasets/2015_2019.csv")

df2020_2024 = pd.read_csv(current_directory + "/Datasets/2020_2024.csv")

In [243]:
df2020_2024.head()

,time,temperature_2m_mean (°C),temperature_2m_max (°C),temperature_2m_min (°C),apparent_temperature_mean (°C),apparent_temperature_max (°C),apparent_temperature_min (°C),precipitation_sum (mm),rain_sum (mm),snowfall_sum (cm),...,wind_direction_10m_dominant (°),winddirection_10m_dominant (°),wind_gusts_10m_mean (km/h),wind_speed_10m_mean (km/h),wind_gusts_10m_min (km/h),wind_speed_10m_min (km/h),relative_humidity_2m_mean (%),relative_humidity_2m_max (%),relative_humidity_2m_min (%),sunshine_duration (s)
0,2020-01-01,-1.4,1.0,-4.4,-6.8,-4.9,-8.7,0.2,0.0,0.14,...,252,252,31.7,16.6,14.4,6.9,67,78,50,9334.95
1,2020-01-02,3.6,8.3,-1.6,-1.1,2.6,-6.8,0.0,0.0,0.00,...,217,217,30.0,16.1,20.5,10.5,68,81,49,25522.97
2,2020-01-03,3.8,8.4,-0.7,0.5,4.6,-4.2,0.0,0.0,0.00,...,266,266,17.5,9.2,10.8,6.1,85,95,66,16050.96
3,2020-01-04,0.9,3.0,-0.4,-3.0,-1.2,-4.0,1.9,0.0,1.33,...,330,330,17.3,8.8,9.4,4.0,74,90,59,5190.75
4,2020-01-05,-0.2,2.0,-1.6,-4.8,-2.8,-6.5,2.8,0.0,1.96,...,266,266,24.0,12.3,15.1,8.0,71,91,56,7100.75


Data Preprocessing

In [244]:
df2010_2014.isnull().sum()

time                               0
temperature_2m_mean (°C)           0
temperature_2m_max (°C)            0
temperature_2m_min (°C)            0
apparent_temperature_mean (°C)     0
apparent_temperature_max (°C)      0
apparent_temperature_min (°C)      0
precipitation_sum (mm)             0
rain_sum (mm)                      0
snowfall_sum (cm)                  0
precipitation_hours (h)            0
wind_speed_10m_max (km/h)          0
wind_gusts_10m_max (km/h)          0
wind_direction_10m_dominant (°)    0
winddirection_10m_dominant (°)     0
wind_gusts_10m_mean (km/h)         0
wind_speed_10m_mean (km/h)         0
wind_gusts_10m_min (km/h)          0
wind_speed_10m_min (km/h)          0
relative_humidity_2m_mean (%)      0
relative_humidity_2m_max (%)       0
relative_humidity_2m_min (%)       0
sunshine_duration (s)              0
dtype: int64

In [245]:
df2015_2019.isnull().sum()

time                               0
temperature_2m_mean (°C)           0
temperature_2m_max (°C)            0
temperature_2m_min (°C)            0
apparent_temperature_mean (°C)     0
apparent_temperature_max (°C)      0
apparent_temperature_min (°C)      0
precipitation_sum (mm)             0
rain_sum (mm)                      0
snowfall_sum (cm)                  0
precipitation_hours (h)            0
wind_speed_10m_max (km/h)          0
wind_gusts_10m_max (km/h)          0
wind_direction_10m_dominant (°)    0
winddirection_10m_dominant (°)     0
wind_gusts_10m_mean (km/h)         0
wind_speed_10m_mean (km/h)         0
wind_gusts_10m_min (km/h)          0
wind_speed_10m_min (km/h)          0
relative_humidity_2m_mean (%)      0
relative_humidity_2m_max (%)       0
relative_humidity_2m_min (%)       0
sunshine_duration (s)              0
dtype: int64

In [246]:
df2020_2024.isnull().sum()

time                               0
temperature_2m_mean (°C)           0
temperature_2m_max (°C)            0
temperature_2m_min (°C)            0
apparent_temperature_mean (°C)     0
apparent_temperature_max (°C)      0
apparent_temperature_min (°C)      0
precipitation_sum (mm)             0
rain_sum (mm)                      0
snowfall_sum (cm)                  0
precipitation_hours (h)            0
wind_speed_10m_max (km/h)          0
wind_gusts_10m_max (km/h)          0
wind_direction_10m_dominant (°)    0
winddirection_10m_dominant (°)     0
wind_gusts_10m_mean (km/h)         0
wind_speed_10m_mean (km/h)         0
wind_gusts_10m_min (km/h)          0
wind_speed_10m_min (km/h)          0
relative_humidity_2m_mean (%)      0
relative_humidity_2m_max (%)       0
relative_humidity_2m_min (%)       0
sunshine_duration (s)              0
dtype: int64

In [247]:
df2020_2024.dtypes

time                                object
temperature_2m_mean (°C)           float64
temperature_2m_max (°C)            float64
temperature_2m_min (°C)            float64
apparent_temperature_mean (°C)     float64
apparent_temperature_max (°C)      float64
apparent_temperature_min (°C)      float64
precipitation_sum (mm)             float64
rain_sum (mm)                      float64
snowfall_sum (cm)                  float64
precipitation_hours (h)            float64
wind_speed_10m_max (km/h)          float64
wind_gusts_10m_max (km/h)          float64
wind_direction_10m_dominant (°)      int64
winddirection_10m_dominant (°)       int64
wind_gusts_10m_mean (km/h)         float64
wind_speed_10m_mean (km/h)         float64
wind_gusts_10m_min (km/h)          float64
wind_speed_10m_min (km/h)          float64
relative_humidity_2m_mean (%)        int64
relative_humidity_2m_max (%)         int64
relative_humidity_2m_min (%)         int64
sunshine_duration (s)              float64
dtype: obje

In [248]:
# converting datatype of time column from object to datetime
df2010_2014['time'] = pd.to_datetime(df2010_2014['time'])
df2015_2019['time'] = pd.to_datetime(df2015_2019['time'])
df2020_2024['time'] = pd.to_datetime(df2020_2024['time'])

Calling the summary generator function

In [249]:
models = ["llama3", "gemma", "mistral"]
dataframes = [df2010_2014, df2015_2019, df2020_2024]

llm_baseline = LlmBaseline(models, dataframes)
summaries = llm_baseline.generator()
    

In [255]:
# print([(item['summaries']) for item in summaries])
summaries[0]['summaries']

[{'model': 'llama3',
  'summary': 'Here is a brief, fluent, and coherent single-paragraph summary:\n\nThe weather conditions during this period were quite varied. Temperatures ranged from -1.9°C to 7.8°C, with most days being around average or slightly below. There was some precipitation, including snowfall on one day, but overall the amount of rainfall was minimal. Wind speeds were moderate, reaching up to 36 km/h on some days, and wind direction varied between different dominant directions throughout the period. The relative humidity remained mostly stable, ranging from 39% to 93%.'},
 {'model': 'gemma',
  'summary': 'The provided dataset records weather data over a period of several days. The temperatures ranged from a low of -8.9°C to a high of 7.8°C, with an average of -2.3°C. Rainfall was limited, occurring only on a few occasions with a total accumulation of 0.42mm. Wind speeds were moderate, with a maximum gust of 49.7km/h recorded. The dominant wind direction varied throughout

In [ ]:
llm_baseline.save_summaries()